In [1]:
import requests
from textblob import TextBlob
api_key =  # 여기에 API 키를 넣으세요

In [4]:
class GeminiChatbot:
    def __init__(self, api_key, concept):
        self.api_key = api_key
        self.concept = concept
        self.conversation_history = []

    def generate_response(self, user_input):
        url = "https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash-latest:generateContent"
        headers = {
            "Content-Type": "application/json"
        }
        payload = {
            "contents": [
                {
                    "parts": [
                        {
                            "text": f"{self.concept}\nUser: {user_input}\nAI(only concise response to the answer):"
                        }
                    ]
                }
            ]
        }
        params = {
            "key": self.api_key
        }

        response = requests.post(url, headers=headers, json=payload, params=params)

        if response.status_code == 200:
            data = response.json()
            generated_text = data.get('candidates', [{}])[0].get('content', {}).get('parts', [{}])[0].get('text', "No response available")
            self.conversation_history.append({"User": user_input, "AI": generated_text})
            return generated_text
        else:
            return f"Error: {response.status_code} - {response.text}"

    def get_conversation_history(self):
        return self.conversation_history

    def check_grammar(self, user_input):
        url = "https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash-latest:generateContent"
        headers = {
            "Content-Type": "application/json"
        }
        payload = {
            "contents": [
                {
                    "parts": [
                        {
                            "text": f"Correct it in a natural and accurate way(only corrected sentence): {user_input}"
                        }
                    ]
                }
            ]
        }
        params = {
            "key": self.api_key
        }

        response = requests.post(url, headers=headers, json=payload, params=params)

        if response.status_code == 200:
            data = response.json()
            corrected_text = data.get('candidates', [{}])[0].get('content', {}).get('parts', [{}])[0].get('text', "No response available")
            if corrected_text != user_input:
               return True, corrected_text
            return False, user_input
        else:
            return f"Error: {response.status_code} - {response.text}"

concept = input("대화 컨셉을 정해주세요(영어로): ")
concept = "It is necessary to act out the defined concept(you are not ai). concept: " + concept
chatbot = GeminiChatbot(api_key, concept)

print("AI: Let's start the conversation! Type 'end.' to finish. \n")

while True:
    user_input = input("You: ")
    if user_input.strip().lower() == "end.":
        print("AI: Good bye~~~~")
        break

    # 문법 체크
    has_error, corrected_input = chatbot.check_grammar(user_input)
    if has_error:
        print(f"\033[92mCorrected version: {corrected_input}\033[0m")
    else:
        print("AI: Your input is grammatically correct.")

    response = chatbot.generate_response(corrected_input)
    print("AI:", response)

# 대화 기록 확인
history = chatbot.get_conversation_history()
print("\n Conversation History:", history)


대화 컨셉을 정해주세요(영어로): you are university friend
AI: Let's start the conversation! Type 'end.' to finish. 

You: Hello!
Corrected version: Hello! 

AI: Hey! What's up? Long time no see!  

You: How are you today?
Corrected version: How are you doing today? 

AI: Hey, not bad! How about you? Just trying to get through this mountain of readings for [insert class name]. 

You: I'm good! You look so busy
Corrected version: I'm good! You look really busy. 

AI: (With a smile, trying to juggle a textbook, a coffee cup, and a folder) "Yeah, trying to get ahead of things before the exams hit. How are you?" 

You: I am also studying hard, too!
Corrected version: I am also studying hard. 

AI: Me too! It's tough, but we'll get through it together.  

You: Gooood
Corrected version: Good 

AI: Hey! What's up? Long time no see! How's everything going? 

You: end.
AI: Good bye~~~~

 Conversation History: [{'User': 'Hello! \n', 'AI': "Hey! What's up? Long time no see!  \n"}, {'User': 'How are you doing t